In [13]:
import numpy as np
import matplotlib.pyplot as plt
import time
from numba import jit, njit, prange, set_num_threads

In [37]:
N = 10000
np.random.seed(111022544)
pos = np.random.randn(N,3)
np.random.seed(111022544)
vel = np.random.randn(N,3)

@njit(parallel=True)
def G_Force(m, Xi,Xj,Yi,Yj,Zi,Zj, G=1, r_soft=0.01):
    
    Xij = Xj - Xi
    Yij = Yj - Yi
    Zij = Zj - Zi
    
    '''
    Xi  = np.outer(np.ones((N,)),pos[:,0])
    Yi  = np.outer(np.ones((N,)),pos[:,1])
    Zi  = np.outer(np.ones((N,)),pos[:,2])
    
    Xij = Xi.T - Xi
    Yij = Yi.T - Yi
    Zij = Zi.T - Zi
    '''

    R = np.sqrt(Xij**2+Yij**2+Zij**2)+r_soft
    F = G*m**2/R**3
    
    Fx = F*Xij
    Fy = F*Yij
    Fz = F*Zij
    
    Fx_total = np.sum(Fx, axis=1)
    Fy_total = np.sum(Fy, axis=1)
    Fz_total = np.sum(Fz, axis=1)
    
    return Fx_total, Fy_total, Fz_total

set_num_threads(8)
#pos = np.array([[0,0,0],[1,1,1]])
t0 = time.time()
Xi,Xj = np.meshgrid(pos[:,0], pos[:,0], indexing='ij')
Yi,Yj = np.meshgrid(pos[:,1], pos[:,1], indexing='ij')
Zi,Zj = np.meshgrid(pos[:,2], pos[:,2], indexing='ij')

Fx_total, Fy_total, Fz_total = G_Force(1, Xi,Xj,Yi,Yj,Zi,Zj)
t1 = time.time()
print(Fx_total) #0.18915488 -0.18915488
print(Fy_total) #0.18915488 -0.18915488
print(Fz_total) #0.18915488 -0.18915488
print('total time',t1-t0)


[-493.68275866  -51.11255907  202.16665181 ...  643.94725799 1198.52751254
  440.85683443]
[-1460.05715724  -467.09827586 -1441.25178345 ...   137.87849003
  -792.22251754 -1665.49387317]
[  961.8335539   -947.33482936 -1367.113171   ...  1057.36124058
   631.08404045 -1051.9727533 ]
total time 4.017325162887573


In [4]:
pos = np.array([[0],[1],[2],[3]])
Xi  = np.outer(np.ones((4,)),pos[:,0])
print(Xi)

xi,xj = np.meshgrid(pos[:,0], pos[:,0], indexing='xy')
print(Xi.T-Xi)

[[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]
[[ 0. -1. -2. -3.]
 [ 1.  0. -1. -2.]
 [ 2.  1.  0. -1.]
 [ 3.  2.  1.  0.]]


In [24]:
1/(np.sqrt(3)+0.01)**3

0.18915487922589125

In [21]:
1/2.01**3*2

0.2462871898274525

In [17]:
#mac outer
37.166444063186646 #N=10000 no jit
12.239506006240845 #N=10000 with jit
#12900k outer
8.607457876205444 #N=10000 no jit
8.322943449020386 #N=10000 with jit
#12900k meshgrid
7.489137887954712 #N=10000 no jit
6.85585355758667  #N=10000 with jit
#12900k meshgrid outside def
5.434570550918579 #N=10000 with jit first
#12900k meshgrid outside def
4.00260066986084 #N=10000 with njit(parallel=True)

12.239506006240845